# 🎓 GCP AIエージェント基盤 - Python基礎とバックエンド理解

このノートブックでは、バックエンドのPythonコードを理解するために必要な基礎知識を学びます。

**対象者**: Python 2ヶ月レベルの初心者

---

## 1. 辞書（dict）の理解

バックエンドでは「辞書」をたくさん使います。
ユーザー情報やAPIレスポンスなど、データの受け渡しに使います。

In [ ]:
# ユーザー情報を辞書で表現
user_info = {
    "uid": "abc123",
    "email": "yamada@example.com",
    "customer_id": "acme-corp"
}

# 値を取り出す
print("UID:", user_info["uid"])
print("Email:", user_info["email"])

# .get() を使うと、キーがなくてもエラーにならない
print("名前:", user_info.get("name", "未設定"))  # "name"がないので"未設定"が返る

### 🔍 実際のコードで確認

auth.pyでは以下のように使われています:

```python
def authenticate_request(request) -> dict:
    ...
    return {
        "uid": "...",
        "email": "...",
        "customer_id": "..."
    }
```

---

## 2. 例外処理（try-except）

エラーが起きても止まらないようにする仕組みです。

In [ ]:
# 例外処理なし → エラーで止まる
def divide_no_try(a, b):
    return a / b

# 例外処理あり → エラーをキャッチして続行
def divide_with_try(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return "ゼロで割ることはできません"
    except Exception as e:
        return f"予期せぬエラー: {e}"

# 実行してみよう
print(divide_with_try(10, 2))   # 正常: 5.0
print(divide_with_try(10, 0))   # エラーをキャッチ

### 🔍 実際のコードで確認

auth.pyでは以下のように使われています:

```python
def verify_token(id_token: str, check_revoked: bool = True) -> dict:
    try:
        # check_revoked=True: ログアウト済みのトークンも拒否
        return auth.verify_id_token(id_token, check_revoked=check_revoked)
    except auth.RevokedIdTokenError:
        raise ValueError("セッションが無効です。再度ログインしてください。")
    except auth.ExpiredIdTokenError:
        raise ValueError("セッションの有効期限が切れました。")
    except auth.InvalidIdTokenError:
        raise ValueError("認証トークンが無効です。")
    except Exception as e:
        raise ValueError(f"トークンの検証に失敗しました: {str(e)}")
```

`raise` は「自分でエラーを発生させる」という意味です。
エラーの種類ごとに違うメッセージを返すことで、ユーザーに適切な対処法を伝えられます。

---

## 3. デコレーター（@）

`@` から始まる行は「デコレーター」といいます。
関数に追加機能をつけます。

In [ ]:
# デコレーターの簡単な例
def my_decorator(func):
    def wrapper():
        print("=== 関数の前 ===")
        func()
        print("=== 関数の後 ===")
    return wrapper

# デコレーターを使う
@my_decorator
def say_hello():
    print("こんにちは！")

# 実行
say_hello()

### 🔍 実際のコードで確認

main.pyでは以下のように使われています:

```python
@app.route("/chat", methods=["POST"])
def chat():
    ...
```

これは「/chat へのPOSTリクエストが来たら、この関数を実行する」という意味です。

---

## 4. async / await（非同期処理）

AIの返答を待っている間、他の処理ができる仕組みです。

In [ ]:
import asyncio

# 普通の関数（同期）
def sync_function():
    return "同期処理の結果"

# 非同期関数
async def async_function():
    await asyncio.sleep(1)  # 1秒待つ（この間、他の処理ができる）
    return "非同期処理の結果"

# Google Colabで非同期関数を実行
result = await async_function()
print(result)

### 🔍 実際のコードで確認

sample_agent/agent.pyでは以下のように使われています:

```python
async def run(self, user_input: str, thread_id: str):
    async for chunk in self.graph.astream(...):
        yield chunk  # 少しずつ結果を返す
```

---

## 5. ジェネレーター（yield）

`yield` は「少しずつ値を返す」仕組みです。
ストリーミング（AIが少しずつ返答する）に使います。

In [ ]:
# 普通の関数（一度に全部返す）
def get_numbers_list():
    return [1, 2, 3, 4, 5]

# ジェネレーター（少しずつ返す）
def get_numbers_generator():
    for i in range(1, 6):
        print(f"  {i}を生成中...")
        yield i  # ここで一度処理が止まり、値を返す

print("=== ジェネレーターの実行 ===")
for num in get_numbers_generator():
    print(f"受け取った値: {num}")

### 🔍 実際のコードで確認

main.pyでは以下のように使われています:

```python
def generate():
    async def async_generate():
        try:
            for chunk in agent.run(message, thread_id):
                yield f"data: {chunk}\n\n"  # 少しずつ返す
            yield "data: [DONE]\n\n"
        except Exception as e:
            # エラー詳細はログに記録、クライアントには汎用メッセージ
            logger.exception("ストリーミング中にエラーが発生")
            yield "data: [ERROR] エラーが発生しました。\n\n"
```

**ポイント**: エラー詳細をクライアントに返さないことで、情報漏洩を防いでいます。

---

## 6. クラスの基礎

バックエンドではクラスを使ってAIエージェントを定義しています。

In [ ]:
# 基底クラス
class BaseAgent:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f"こんにちは、{self.name}です"

# 継承したクラス
class SampleAgent(BaseAgent):
    def __init__(self, name, specialty):
        super().__init__(name)  # 親クラスの__init__を呼ぶ
        self.specialty = specialty

    def introduce(self):
        return f"{self.greet()}。専門は{self.specialty}です。"

# 使ってみる
agent = SampleAgent("AI助手", "質問への回答")
print(agent.introduce())

---

## 📝 まとめ

| 概念 | 説明 | 使用箇所 |
|------|------|----------|
| 辞書（dict） | キーと値のペア | ユーザー情報、API レスポンス |
| 例外処理（try-except） | エラーをキャッチ | 認証、API呼び出し |
| デコレーター（@） | 関数に追加機能 | APIエンドポイント定義 |
| async/await | 非同期処理 | AIへの問い合わせ |
| yield | 少しずつ返す | ストリーミングレスポンス |
| クラス | オブジェクト設計 | エージェント定義 |
| 正規表現（re） | 文字列パターン検証 | 入力値のバリデーション |
| logging | ログ出力 | エラー記録（情報漏洩防止） |

これらを理解すると、バックエンドのコードが読めるようになります！

### 🔒 セキュリティのポイント

本番コードでは以下のセキュリティ対策が実装されています:

- **入力検証**: メッセージ長制限（10,000文字）、thread_id形式検証
- **トークン検証強化**: 失効チェック（`check_revoked=True`）
- **エラー情報保護**: 内部エラーの詳細をクライアントに返さない
- **依存関係固定**: 脆弱性のあるバージョンを避けるため固定バージョン指定

---

## 🚀 次のステップ

1. `backend/src/main.py` を読んでみる
2. `backend/src/common/auth.py` を読んでみる
3. 実際にローカルでサーバーを動かしてみる